In [1]:
import pandas as pd
import numpy as np
from processing_functions.tf_idf_functions import calculate_tfidf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
tqdm.pandas()

In [2]:
def split_test_sets(df, test_size=0.2, temporal_test_size=75, random_state=42):
    """
    Splits the dataset into training, validation, temporal test, and thrower test sets.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data.
    - test_size (float, optional): The proportion of the dataset to include in the validation set. Default is 0.2.
    - temporal_test_size (int, optional): The number of games to include in the temporal test set. Default is 75.
    - random_state (int, optional): Random seed for reproducibility. Default is 42.
    
    Returns:
    - train_df (pd.DataFrame): Training set.
    - val_df (pd.DataFrame): Validation set.
    - temporal_test_df (pd.DataFrame): Temporal test set.
    - thrower_test_df (pd.DataFrame): Hold-out test set for specific throwers.
    """
    # Select 50 throwers with > 200 throws for hold-out-testing
    throwers = df.groupby('thrower').thrower.count()
    hold_out_throwers = np.array(throwers[throwers > 200].index)
    np.random.seed(random_state)
    hold_out_throwers = np.random.choice(hold_out_throwers, size=50, replace=False)
    thrower_test_df = df[df.thrower.isin(hold_out_throwers)].copy()
    df_filtered = df[~df.thrower.isin(hold_out_throwers)].copy()
    
    # Extract date and year
    df_filtered.loc[:,'gameDate'] = pd.to_datetime(df_filtered.loc[:,'gameID'].str[:10], format='%Y-%m-%d')
    df_filtered.loc[:,'year'] = df_filtered.loc[:,'gameDate'].dt.year
    df_sorted = df_filtered.sort_values(by='gameDate')
    
    # Select 75 most recent games for hold-out-testing 
    unique_game_ids = df_sorted.loc[:,'gameID'].unique()
    temporal_test_ids = unique_game_ids[-temporal_test_size:]
    
    # In remeaining data, split into test and train. Make sure the hold out testing set is balanced between years
    remaining_ids = unique_game_ids[:-temporal_test_size]
    total_remaining = len(remaining_ids)
    num_games_per_year = int(total_remaining * test_size / 4)
    remaining_df = df_sorted[df_sorted.loc[:,'gameID'].isin(remaining_ids)]
    val_games = []
    for year, group in remaining_df.groupby('year'):
        sampled_games = group.sample(n=num_games_per_year, random_state=random_state)
        val_games.append(sampled_games.loc[:,'gameID'].unique())
    
    val_ids = np.concatenate(val_games)
    
    # create final dfs
    train_ids = [game_id for game_id in remaining_ids if game_id not in val_ids]
    train_df = df_filtered[df_filtered.loc[:,'gameID'].isin(train_ids)]
    val_df = df_filtered[df_filtered.loc[:,'gameID'].isin(val_ids)]
    temporal_test_df = df_filtered[df_filtered.loc[:,'gameID'].isin(temporal_test_ids)]
    
    return train_df, val_df, temporal_test_df, thrower_test_df

In [3]:
def initialize_data(filepath='./data/processed/all_games_0926.csv'):
    """
    Initializes the dataset by loading a CSV file, processing the data to calculate various columns
    like goal, outcome, point_outcome, throw_distance, and completion. The function handles missing values
    and adjusts the 'times' column based on certain time limits.
    
    Parameters:
    - filepath (str): The path to the CSV file containing the game data.
    
    Returns:
    - pd.DataFrame: The processed DataFrame with calculated columns.
    """
    df = pd.read_csv(filepath)
    df = df.dropna(subset=['thrower_x', 'thrower_y'])
    df.loc[:,'goal'] = (df.loc[:,'receiver_y'] > 100) & (df.loc[:,'turnover'] == 0)
    df.loc[:,'outcome'] = 0  # 0 if possession ended without a goal otherwise 1
    df.loc[:,'point_outcome'] = 0 # 0 if point ended without a goal otherwise 1
    df.loc[:,'throw_distance'] = np.sqrt((df.loc[:,'receiver_x'] - df.loc[:,'thrower_x'])**2 + (df.loc[:,'receiver_y'] - df.loc[:,'thrower_y'])**2)
    df['x_diff'] = df['receiver_x'] - df['thrower_x']
    df['y_diff'] = df['receiver_y'] - df['thrower_y']
    df['throw_angle'] = np.abs((np.degrees(np.arctan2(df['y_diff'], df['x_diff'])) + 90) % 360 - 180)
    df.loc[:,'completion'] = 1 - df.loc[:,'turnover']
    df.loc[:,'gameDate'] = pd.to_datetime(df.loc[:,'gameID'].str[:10], format='%Y-%m-%d')
    df.loc[:,'year'] = df.loc[:,'gameDate'].dt.year

    # get hockey assist by checking if previous receiver is the same as assist thrower
    df['prev_receiver'] = df['receiver'].shift(1)
    df['prev_thrower'] = df['thrower'].shift(1)
    hockey_assists_df = df[
        (df['receiver_y'] > 100) & (df['turnover'] == 0) & 
        (df['thrower'] == df['prev_receiver'])
    ]

    df['hockey_assist'] = 0  
    df.loc[hockey_assists_df.index - 1, 'hockey_assist'] = 1  
    df.drop(['prev_receiver', 'prev_thrower'], axis=1, inplace=True)

    # convert times to minutes and show time left in quarter
    df.times = (df.times / 60)
    df.loc[df['times'] < 0, 'times'] = df.loc[df['times'] < 0, 'times'] + 5
    while len(df.times[df.times > 12]) > 0:
        df.loc[df['times'] > 12, 'times'] = df.loc[df['times'] > 12, 'times'] - 12
    # there are no times for double OT so make it a full quarter time throughout
    df.loc[df['game_quarter'] == 6, 'times'] = 12

    # define outcome and point outcome
    for _, group in df.groupby(['gameID', 'home_team_score', 'away_team_score', 'possession_num', 'game_quarter']):
        last_throw = group.iloc[-1]  
        if last_throw['receiver_y'] > 100 and last_throw['turnover'] == 0:
            df.loc[group.index, 'outcome'] = 1  
    for _, group in df.groupby(['gameID', 'home_team_score', 'away_team_score', 'game_quarter']):
        last_throw = group.iloc[-1]  # Get the last row in the group
        if last_throw['receiver_y'] > 100 and last_throw['turnover'] == 0:
            mask = group['is_home_team'] == last_throw['is_home_team']
            df.loc[group[mask].index, 'point_outcome'] = 1
    return df

In [4]:
def get_completion_percentage(df, test_dfs=None):
    """
    Calculate the completion percentage for each thrower and add it to the DataFrame.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing throw data with a 'thrower' and 'completion' column.
    - test_dfs (list of pd.DataFrame, optional): List of test DataFrames to which the completion percentage will also be added.
    
    Returns:
    - train_df (pd.DataFrame): DataFrame with completion percentage added.
    - test_dfs_final (list of pd.DataFrame): List of test DataFrames with completion percentage added (if provided).
    """
    
    # Group by 'thrower' and calculate successful completions and total throws
    completion_stats = df.groupby('thrower').agg(
        successful_completions=('completion', 'sum'),  # Total successful completions per thrower
        total_throws=('completion', 'count')  # Total number of throws per thrower
    ).reset_index()

    #  Calculate completion percentage for each thrower
    completion_stats['completion_percentage'] = (
        completion_stats['successful_completions'] / completion_stats['total_throws']
    ) * 100
    new_rows = completion_stats[['thrower', 'completion_percentage']].rename(
        columns={'completion_percentage': 'thrower_completion_percentage'}
    )
    train_df = df.merge(new_rows, on='thrower', how='left').sort_values('thrower_completion_percentage')

    test_dfs_final = []
    if test_dfs is not None:
        # For each test DataFrame, merge completion percentage and fill missing values with the median
        for test_df in test_dfs:
            test_df = test_df.merge(new_rows, on='thrower', how='left').sort_values('thrower_completion_percentage')
            test_df.thrower_completion_percentage = test_df.thrower_completion_percentage.fillna(
                test_df.thrower_completion_percentage.median()
            )
            test_dfs_final.append(test_df)

    return train_df, test_dfs_final


In [5]:
## processing to create data splits and calculate player features from only train data
## used for model training, showing generalizability and predictability
filepath='../data/all_games_1024.csv'
df = initialize_data(filepath)

train_df, test_df_random, test_df_time, test_df_thrower = split_test_sets(df, random_state=0, test_size=0.4)
train_df, test_dfs = get_completion_percentage(df, test_dfs=[test_df_random, test_df_time, test_df_thrower])
joblib.dump({'train_df':train_df, 'test_df_random':test_df_random, 'test_df_time':test_df_time, 'test_df_thrower':test_df_thrower}, '../data/processed/data_splits_1003.jblb')

['../data/processed/data_splits_1003.jblb']

In [6]:
## processing for all throws into a single dataframe
## used for descriptive purposes, meta analytics and derived metrics

filepath='../data/all_games_1024.csv'
df = initialize_data(filepath)
df, _ = get_completion_percentage(df)
joblib.dump({'df':df}, '../data/processed/data_1003.jblb')

['../data/processed/data_1003.jblb']

In [7]:
df

,thrower,thrower_x,thrower_y,receiver,receiver_x,receiver_y,turnover,possession_num,possession_throw,game_quarter,...,point_outcome,throw_distance,x_diff,y_diff,throw_angle,completion,gameDate,year,hockey_assist,thrower_completion_percentage
213828,aseibert,15.80,45.15,NaN,1.74,48.95,1,1,3,3,...,0,14.564464,-14.06,3.80,74.875993,0,2021-06-19,2021,0,0.0
37999,mgoetsch,0.93,58.45,NaN,1.70,53.93,1,1,6,1,...,0,4.585117,0.77,-4.52,170.332244,0,2023-07-21,2023,0,0.0
268706,ejoyce,17.79,93.90,NaN,15.47,109.54,1,1,7,1,...,0,15.811135,-2.32,15.64,8.437590,0,2022-06-11,2022,0,0.0
243794,dbaker1,1.67,47.76,alopezesc,1.67,47.76,1,2,3,2,...,0,0.000000,0.00,0.00,90.000000,0,2022-05-21,2022,0,0.0
221402,rjones,-23.06,52.96,bobryan,-24.41,57.09,1,1,3,4,...,0,4.345043,-1.35,4.13,18.101353,0,2021-08-29,2021,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76798,bthomsen,23.24,46.52,dferriter,16.24,49.15,0,1,1,4,...,0,7.477760,-7.00,2.63,69.408083,1,2023-07-25,2023,0,100.0
76809,cnelson,-9.33,90.07,wanderson,-4.89,97.35,0,1,2,4,...,1,8.527133,4.44,7.28,31.378602,1,2023-07-25,2023,1,100.0
76824,msutton,-12.10,66.62,wgoal,-16.76,76.74,0,1,1,4,...,0,11.141364,-4.66,10.12,24.724856,1,2023-07-25,2023,0,100.0
82751,ctexeira,-16.80,37.29,edavis,-22.35,32.38,0,1,4,4,...,1,7.410169,-5.55,-4.91,131.498696,1,2024-07-21,2024,0,100.0
